# Uwaga

Korzystam jedynie z dwóch modeli, a nie z trzech jak jest to sformułowane w treści zadania.

Wynika to z faktu, że jedynie 2 z pośród 6, które próbowałem, udało mi się uruchomić. Pozostałe, takie jak: `google/mt5-base`, `allegro/plt5-base` wymagały zależności natywnych (`protobuf`), których nie mogłem zainstalować system-wide jako, że pracowałem
na sprzęcie na którym nie miałem `roota`. 

Popatrzę na zajęciach na wnioski innych, natomiast w tym sprawozdaniu będą one oparte tylko na testach przeprowadzonych na dwóch modelach: `papuGaPT2`, `allegro/herbert-base-cased`. "W zamian" nieco więcej eksperymentuję na końcu. 

# Zadanie 2


Download three Polish models from the Huggingface repository. These should be regular language models, which were not fine-tuned. E.g. HerBERT and papuGaPT2 are good examples.

In [1]:
from transformers import pipeline, AutoTokenizer

modelname_allegro_herbert_base_cased = "allegro/herbert-base-cased"
modelname_papugapt2 = "flax-community/papuGaPT2"
modelname_google_mt5_base = "google/mt5-base"
modelname_plt5 = "allegro/plt5-base"

/home/kkafara/.cache/virtualenvs/python-3.11-venv-ds/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
papuga_pipe = pipeline(model=modelname_papugapt2)

In [3]:
herbert_pipe = pipeline(model=modelname_allegro_herbert_base_cased)

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.sso.sso_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.sso.sso_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# mt5_pipe = pipeline(model=modelname_google_mt5_base)

# Zadanie 3

Devise a method to test if the langage model understands Polish cases. E.g. testing for nominal case could be expressed as "Warszawa to największe [MASK]",
and the masked word should be in nominative case. Create sentences for each case.


*Zdaję sobie sprawę, że poniżej porównuję różne modele, na różnych zadaniach, ale nie mogłem tak dobrać, żeby wszystkie trzy obsługiwały te same zadania & wyniki były sensowne*. Do tego dokumentacja większości polskich modeli
jest tragiczna (mówię tutaj w szczególności o HerBERT, którego docsy są co najwyżej prowizoryczne i absolutinie nie begginer-friendly (mówię o sobie)).

In [14]:
# crashuje w przypadku tego modelu
# papuga_fm = pipeline(task="fill-mask", model=modelname_papugapt2)

papuga_tg = pipeline(task="text-generation", model=modelname_papugapt2)
herbert_fm = pipeline(task="fill-mask", model=modelname_allegro_herbert_base_cased)

### Mianownik

In [15]:
papuga_tg("Stolicą Polski oczywiście jest")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Stolicą Polski oczywiście jest Warszawa, bo w niej mamy dostęp do wszystkich najważniejszych miejsc i zabytków, które są w Polsce warte zobaczenia. A na pewno Polska jest miejscem, do którego warto przyjechać, jeśli szukamy jakiegoś miejsca widokowego, w końcu po to'}]

Na wiele różnych (~10) uruchomień powyższej komórki poprawną odpowiedź (Warszawa) otrzymałem tylko 2 razy. Niemniej jednak forma samego wyrazu była zawsze poprawna.

Zwracam uwagę też na to, że w promptach nie korzystam z `[MASK]`, ponieważ przykłady w dokumentacji modelu `papuGaPT2` tego nie robią (podejrzewam, że nie ma możliwości maskowania, bo potrafi tylko
"kontynuować zdania", ale nie mam pewności). Nie wiem też jaki to ma wpływ na wyniki.

Fragment dokumentacji do którego się odnoszę: https://huggingface.co/flax-community/papuGaPT2?text=Najsmaczniejszy+polski+owoc+to#text-generation

In [16]:
herbert_fm("Stolicą Polski oczywiścje jest <mask>")

[{'score': 0.36618852615356445,
  'token': 5936,
  'token_str': 'Warszawa',
  'sequence': 'Stolicą Polski oczywiścje jest Warszawa'},
 {'score': 0.16102533042430878,
  'token': 4394,
  'token_str': 'Kraków',
  'sequence': 'Stolicą Polski oczywiścje jest Kraków'},
 {'score': 0.12439846247434616,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Stolicą Polski oczywiścje jest …'},
 {'score': 0.040639717131853104,
  'token': 6999,
  'token_str': 'Poznań',
  'sequence': 'Stolicą Polski oczywiścje jest Poznań'},
 {'score': 0.038738179951906204,
  'token': 9250,
  'token_str': 'Wrocław',
  'sequence': 'Stolicą Polski oczywiścje jest Wrocław'}]

W przypadku tego modelu (i tego zadania" `fill-mask`), odpowiedź poprawna jako pierwsza (najwyższy score) była pokazywana za każdym razem. Forma wyrazu też zawsze była poprawna.

### Dopełniacz

In [17]:
papuga_tg("Nie będziemy palić w piecu dla dobra całej")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Nie będziemy palić w piecu dla dobra całej naszej społeczności. Do tego będziemy palić w obecności i z upoważnienia swoich najbliższych.\nCzy to z naszej zgody, czy z woli, czy życzenia, to oni sami decydują w jak należy z naszej perspektywy wyglądać.\n'}]

Wyraz / sekwencja została uzupełniona wyrazami w prawidłowej formie i jako tako z sensem. 

In [18]:
herbert_fm("Nie będziemy palić w piecu dla dobra całej <mask>")

[{'score': 0.28093963861465454,
  'token': 4877,
  'token_str': 'rodziny',
  'sequence': 'Nie będziemy palić w piecu dla dobra całej rodziny'},
 {'score': 0.15739218890666962,
  'token': 2960,
  'token_str': 'Polski',
  'sequence': 'Nie będziemy palić w piecu dla dobra całej Polski'},
 {'score': 0.09003997594118118,
  'token': 23891,
  'token_str': 'ludzkości',
  'sequence': 'Nie będziemy palić w piecu dla dobra całej ludzkości'},
 {'score': 0.07397733628749847,
  'token': 4813,
  'token_str': 'Europy',
  'sequence': 'Nie będziemy palić w piecu dla dobra całej Europy'},
 {'score': 0.06293289363384247,
  'token': 11849,
  'token_str': 'społeczności',
  'sequence': 'Nie będziemy palić w piecu dla dobra całej społeczności'}]

Tutaj formy wyrazów są jak najbardziej poprawne.

### Celownik

In [20]:
papuga_tg("Nowy karabin maszynowy zostanie wydany każdemu")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Nowy karabin maszynowy zostanie wydany każdemu obywatelowi, nie tylko mieszkańcom USA. Każdy Amerykanin może zapłacić w USA za broń do celów specjalnych, takich jak broń chemiczna lub biologiczna. Amerykański rynek uzbrojony w taki karabin maszynowy będzie'}]

Forma zwykle była poprawna. Ciekawe jest (choć pewnie mało zaskakujące), że gdy przykład jest z bronią, to bardzo często pojawia się USA :D

In [21]:
herbert_fm("Nowy karabin maszynowy zostanie wydany każdemu <mask>")

[{'score': 0.5254270434379578,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Nowy karabin maszynowy zostanie wydany każdemu.'},
 {'score': 0.03937682509422302,
  'token': 1725,
  'token_str': '!',
  'sequence': 'Nowy karabin maszynowy zostanie wydany każdemu!'},
 {'score': 0.03688626363873482,
  'token': 19617,
  'token_str': 'dziecku',
  'sequence': 'Nowy karabin maszynowy zostanie wydany każdemu dziecku'},
 {'score': 0.0337052047252655,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Nowy karabin maszynowy zostanie wydany każdemu …'},
 {'score': 0.032657984644174576,
  'token': 1046,
  'token_str': 'z',
  'sequence': 'Nowy karabin maszynowy zostanie wydany każdemu z'}]

Tutaj model nieco mnie ograł uzupełniając interpunkcją, ale po outputcie z "dziecku" mogę stwierdzić, że dobrał poprawną formę, choć może nie najbardziej sensowną (USA xD)

### Biernik

In [31]:
papuga_tg("Widzę w klatce niesamowicie kolorową", max_new_tokens=20)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Widzę w klatce niesamowicie kolorową papużkę, i jak na moją wyobraźnię może mieć całkiem sporo kolorów :))\nPotrafią zauroczyć'}]

Papuga przewidziała papugę, fantastycznie.

In [36]:
herbert_fm("Widzę w klatce niesamowicie kolorową <mask>")

[{'score': 0.42295607924461365,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Widzę w klatce niesamowicie kolorową.'},
 {'score': 0.065530925989151,
  'token': 22731,
  'token_str': 'skórę',
  'sequence': 'Widzę w klatce niesamowicie kolorową skórę'},
 {'score': 0.038813650608062744,
  'token': 1725,
  'token_str': '!',
  'sequence': 'Widzę w klatce niesamowicie kolorową!'},
 {'score': 0.030762039124965668,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Widzę w klatce niesamowicie kolorową …'},
 {'score': 0.02667396515607834,
  'token': 10319,
  'token_str': 'postać',
  'sequence': 'Widzę w klatce niesamowicie kolorową postać'}]

Pod względem syntaktycznym, semantycznie nieco mniej.

### Narzędnik

In [47]:
papuga_tg("Do Tajwanu pojedziemy z")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Do Tajwanu pojedziemy z jednym z największych, globalnych dystrybutorów produktów z kamienia naturalnego – firmą Kafle Ceramika Sp. z o.o.\nW Tajpej znajdziemy zarówno duże, popularne miasta („Manhaket”, „Papuana”,'}]

In [48]:
herbert_fm("Do Tajwany pojedziemy z <mask>")

[{'score': 0.1072307899594307,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Do Tajwany pojedziemy z …'},
 {'score': 0.04334069415926933,
  'token': 1335,
  'token_str': ':',
  'sequence': 'Do Tajwany pojedziemy z :'},
 {'score': 0.03576905280351639,
  'token': 10424,
  'token_str': 'dziećmi',
  'sequence': 'Do Tajwany pojedziemy z dziećmi'},
 {'score': 0.03332872316241264,
  'token': 13128,
  'token_str': 'dzieckiem',
  'sequence': 'Do Tajwany pojedziemy z dzieckiem'},
 {'score': 0.03192038834095001,
  'token': 6094,
  'token_str': 'Warszawy',
  'sequence': 'Do Tajwany pojedziemy z Warszawy'}]

:like:

### Miejscownik

In [45]:
papuga_tg("Media przez ostatni tydzień nie przestawały mówić o")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Media przez ostatni tydzień nie przestawały mówić o kolejnych protestach górniczych, które zostały rozpoczęte po tym, jak górnicy zdecydowali się na podwyżki płac. Jednocześnie były prowadzone rozmowy na temat wspólnego wyjścia z kopalni i przejścia na węgiel. W marcu media obie'}]

Poprawna forma

In [46]:
herbert_fm("Media przez ostatni tydzień nie przestawyały mówić o <mask>")

[{'score': 0.5119613409042358,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Media przez ostatni tydzień nie przestawyały mówić o …'},
 {'score': 0.013213361613452435,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Media przez ostatni tydzień nie przestawyały mówić o.'},
 {'score': 0.012496136128902435,
  'token': 1335,
  'token_str': ':',
  'sequence': 'Media przez ostatni tydzień nie przestawyały mówić o :'},
 {'score': 0.012240143492817879,
  'token': 21144,
  'token_str': 'problemach',
  'sequence': 'Media przez ostatni tydzień nie przestawyały mówić o problemach'},
 {'score': 0.011556684039533138,
  'token': 39319,
  'token_str': 'kryzysie',
  'sequence': 'Media przez ostatni tydzień nie przestawyały mówić o kryzysie'}]

Słabe są te interpunkcje, ale ostatnie 2 tokeny są w porządku syntaktycznie i semantycznie

### Wołacz

In [51]:
papuga_tg("Litwo!")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Litwo! Ojczyzno moja” był bardzo ciekawy. Nie mogliśmy się nadziwić jakim wielkim znawcą poezji był nasz kolega, czy też pani Grażyna i co o niej wiemy, jakie ma ona dla Polski znaczenie. Ucieszyliśmy się z tego'}]

Nie za bardzo przychodzi mi do głowy przykłąd z wołaczem na końcu zdania :D 

In [53]:
herbert_fm("<mask>! Ojczyzno moja")

[{'score': 0.3647006154060364,
  'token': 14080,
  'token_str': 'Witaj',
  'sequence': 'Witaj! Ojczyzno moja'},
 {'score': 0.051284655928611755,
  'token': 21136,
  'token_str': 'Hej',
  'sequence': 'Hej! Ojczyzno moja'},
 {'score': 0.04865175113081932,
  'token': 1063,
  'token_str': 'O',
  'sequence': 'O! Ojczyzno moja'},
 {'score': 0.04752074554562569,
  'token': 10303,
  'token_str': 'Pozdrawiam',
  'sequence': 'Pozdrawiam! Ojczyzno moja'},
 {'score': 0.04677436500787735,
  'token': 15406,
  'token_str': 'Cześć',
  'sequence': 'Cześć! Ojczyzno moja'}]

Nie za bardzo

# Zadanie 4

Devise a method to test long-range relationships such as gender. E.e. you can use two verbs with masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.

Jako, że `papuGaPT2` nie ma możliwości `fill-mask` (a przynajmniej nie znalazłem), to będę bardziej różnicował przykłady pomiedzy modelami.

In [54]:
papuga_tg("Płotka, klacz Geralta, wcale nie ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Płotka, klacz Geralta, wcale nie owiana tajemnicą, a z oczywistych powodów bardzo rzadka na całej planecie, a i w świecie Gwiezdnych Wojen nie była przedmiotem podziwu i zazdrości wśród zwolenników tego typu gier. I tak'}]

Widzimy, że model "wywnioskował" płeć i konsekwentnie się jej trzyma. Możemy spróbować jeszcze z mniejszą liczbą informacji (bo tutaj jest zarówno "Płotka", jak i "klacz").

In [56]:
papuga_tg("Płotka, jako wierny wierzchowiec Geralta, wcale nie")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Płotka, jako wierny wierzchowiec Geralta, wcale nie jest taki zły.\n„Gawiedź” opowiada też prawdziwą historię rodziny Obi-Wana Kenobi, która nie dość, że nie miała z kim zostawić córki,'}]

Teraz już popłynął. I coś go bardzo ciągnie do "Star Wars".

In [60]:
papuga_tg("Klacz Geralta wcale nie ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Klacz Geralta wcale nie owiana była takim urokiem jak inni, lecz jego ojciec z właściwą sobie lekkomyślnością, mimo braku zainteresowania i chęci do działania, a to wszystko za sprawą swoich dwóch córek!\nNależąca do'}]

^-- powyżej widzimy, że podmiot został już zgubiony, a to zdanie nie jest złożone

In [61]:
herbert_fm("Płotka, klacz Geralta, wcale nie <mask> tego robić, ani tym bardziej przejmować się ")

[{'score': 0.4900616705417633,
  'token': 3583,
  'token_str': 'musi',
  'sequence': 'Płotka, klacz Geralta, wcale nie musi tego robić, ani tym bardziej przejmować się'},
 {'score': 0.0974636971950531,
  'token': 5578,
  'token_str': 'powinna',
  'sequence': 'Płotka, klacz Geralta, wcale nie powinna tego robić, ani tym bardziej przejmować się'},
 {'score': 0.06618716567754745,
  'token': 3633,
  'token_str': 'chce',
  'sequence': 'Płotka, klacz Geralta, wcale nie chce tego robić, ani tym bardziej przejmować się'},
 {'score': 0.05556068569421768,
  'token': 10437,
  'token_str': 'chciała',
  'sequence': 'Płotka, klacz Geralta, wcale nie chciała tego robić, ani tym bardziej przejmować się'},
 {'score': 0.05114469677209854,
  'token': 8301,
  'token_str': 'zamierza',
  'sequence': 'Płotka, klacz Geralta, wcale nie zamierza tego robić, ani tym bardziej przejmować się'}]

^-- Wygląda to solidnie

In [67]:
herbert_fm("Klacz Geralta zreflektowała się szybko i <mask> w jego stronę, natomiast on nie mógł wstać")

[{'score': 0.629269003868103,
  'token': 23086,
  'token_str': 'ruszyła',
  'sequence': 'Klacz Geralta zreflektowała się szybko i ruszyła w jego stronę, natomiast on nie mógł wstać'},
 {'score': 0.12875309586524963,
  'token': 19464,
  'token_str': 'poszła',
  'sequence': 'Klacz Geralta zreflektowała się szybko i poszła w jego stronę, natomiast on nie mógł wstać'},
 {'score': 0.05777115747332573,
  'token': 25094,
  'token_str': 'rzuciła',
  'sequence': 'Klacz Geralta zreflektowała się szybko i rzuciła w jego stronę, natomiast on nie mógł wstać'},
 {'score': 0.023694951087236404,
  'token': 45675,
  'token_str': 'uciekła',
  'sequence': 'Klacz Geralta zreflektowała się szybko i uciekła w jego stronę, natomiast on nie mógł wstać'},
 {'score': 0.023675614967942238,
  'token': 41642,
  'token_str': 'skoczyła',
  'sequence': 'Klacz Geralta zreflektowała się szybko i skoczyła w jego stronę, natomiast on nie mógł wstać'}]

In [65]:
herbert_fm("Klacz Geralta <mask> się szybko i pobiegła w jego stronę, natomiast on nie mógł wstać")

[{'score': 0.579107403755188,
  'token': 17489,
  'token_str': 'zatrzymała',
  'sequence': 'Klacz Geralta zatrzymała się szybko i pobiegła w jego stronę, natomiast on nie mógł wstać'},
 {'score': 0.1394488662481308,
  'token': 40910,
  'token_str': 'podniosła',
  'sequence': 'Klacz Geralta podniosła się szybko i pobiegła w jego stronę, natomiast on nie mógł wstać'},
 {'score': 0.0818558931350708,
  'token': 12142,
  'token_str': 'pojawiła',
  'sequence': 'Klacz Geralta pojawiła się szybko i pobiegła w jego stronę, natomiast on nie mógł wstać'},
 {'score': 0.026939714327454567,
  'token': 23086,
  'token_str': 'ruszyła',
  'sequence': 'Klacz Geralta ruszyła się szybko i pobiegła w jego stronę, natomiast on nie mógł wstać'},
 {'score': 0.015271407552063465,
  'token': 10061,
  'token_str': 'znalazła',
  'sequence': 'Klacz Geralta znalazła się szybko i pobiegła w jego stronę, natomiast on nie mógł wstać'}]

HerBERT tutaj radzi sobie lepiej z formami czasowników. Nawet gdy w zdaniu występuję "drugi potencjalny podmiot". Spróbujmy jeszcze mu skomplikować.

In [89]:
herbert_fm("Kupiłabym Ci prezent, lecz ostatnio zachowujesz się niemile, dlatego nie <mask> ode mnie nic w tym roku, a już na pewno nie podaruję Ci")

[{'score': 0.2289886474609375,
  'token': 13792,
  'token_str': 'chcesz',
  'sequence': 'Kupiłabym Ci prezent, lecz ostatnio zachowujesz się niemile, dlatego nie chcesz ode mnie nic w tym roku, a już na pewno nie podaruję Ci'},
 {'score': 0.13760817050933838,
  'token': 8303,
  'token_str': 'masz',
  'sequence': 'Kupiłabym Ci prezent, lecz ostatnio zachowujesz się niemile, dlatego nie masz ode mnie nic w tym roku, a już na pewno nie podaruję Ci'},
 {'score': 0.08001423627138138,
  'token': 39407,
  'token_str': 'dostałam',
  'sequence': 'Kupiłabym Ci prezent, lecz ostatnio zachowujesz się niemile, dlatego nie dostałam ode mnie nic w tym roku, a już na pewno nie podaruję Ci'},
 {'score': 0.060660675168037415,
  'token': 34355,
  'token_str': 'bierz',
  'sequence': 'Kupiłabym Ci prezent, lecz ostatnio zachowujesz się niemile, dlatego nie bierz ode mnie nic w tym roku, a już na pewno nie podaruję Ci'},
 {'score': 0.04980644956231117,
  'token': 43800,
  'token_str': 'odbierze',
  'sequenc

Tu już się pogubił. Widzimy, że forma czasownika nie jest poprawnie dopasowana do kontekstu: "dostałam", "odbierze", itd.

# Zadanie 5

Check if the model captures real-world knolwedge. For instance a sentence "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza." checks if the model "knows" the description of water. Define at least 3 such sentences.

In [73]:
papuga_tg("Krzysztof Krawczyk był")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Krzysztof Krawczyk był gościem pierwszego odcinka programu “Masters of Computer & Internet” na antenie Dwójki. Zadał tam pytanie, czy chce prowadzić własne badania naukowe. Dziennikarze pytali go, czy uważa, że tak.\n–'}]

Fatalna wiedza... ;)

In [70]:
papuga_tg("Koronacja Napoleona na cesarza miała miejsce drugiego grudnia roku ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Koronacja Napoleona na cesarza miała miejsce drugiego grudnia roku \xad1815, a więc już po śmierci cesarza. Jednakże tego samego dnia Bonaparte przybył do swojego miasta i z powodu wojny, do której zapadł, miasto na kilka lat straciło'}]

No niestety nie można zgodzić się z tymi fantazjami.

In [72]:
herbert_fm("Krzysztof Krawczyk był <mask>")

[{'score': 0.2605268955230713,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Krzysztof Krawczyk był …'},
 {'score': 0.03785598650574684,
  'token': 1019,
  'token_str': 'w',
  'sequence': 'Krzysztof Krawczyk był w'},
 {'score': 0.025451788678765297,
  'token': 8720,
  'token_str': 'najlepszy',
  'sequence': 'Krzysztof Krawczyk był najlepszy'},
 {'score': 0.023693649098277092,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Krzysztof Krawczyk był.'},
 {'score': 0.02130425162613392,
  'token': 1335,
  'token_str': ':',
  'sequence': 'Krzysztof Krawczyk był :'}]

"był najlepszy" <-- z tym możemy się zgodzić ;D Gdyby udało się wyeliminować tą interpunkcję, to byłoby całkiem nieźle

In [74]:
herbert_fm("Koronacja Napoleona na cesarza miałą miejsce drugiego grudnia roku <mask>")

[{'score': 0.768883228302002,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Koronacja Napoleona na cesarza miałą miejsce drugiego grudnia roku.'},
 {'score': 0.018007507547736168,
  'token': 42312,
  'token_str': '1815',
  'sequence': 'Koronacja Napoleona na cesarza miałą miejsce drugiego grudnia roku 1815'},
 {'score': 0.01520566176623106,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Koronacja Napoleona na cesarza miałą miejsce drugiego grudnia roku …'},
 {'score': 0.01120938640087843,
  'token': 40087,
  'token_str': '1863',
  'sequence': 'Koronacja Napoleona na cesarza miałą miejsce drugiego grudnia roku 1863'},
 {'score': 0.011038212105631828,
  'token': 48876,
  'token_str': '1870',
  'sequence': 'Koronacja Napoleona na cesarza miałą miejsce drugiego grudnia roku 1870'}]

No niestety, nie ma tutaj poprawnej odpowiedzi (1804).

Dam jeszcze jedna szansę.

In [75]:
herbert_fm("<mask> Gagarin był pierwszym człowiekiem w kosmosie")

[{'score': 0.04818161949515343,
  'token': 14012,
  'token_str': 'John',
  'sequence': 'John Gagarin był pierwszym człowiekiem w kosmosie'},
 {'score': 0.045256227254867554,
  'token': 18154,
  'token_str': 'George',
  'sequence': 'George Gagarin był pierwszym człowiekiem w kosmosie'},
 {'score': 0.04135838896036148,
  'token': 8269,
  'token_str': 'Aleksander',
  'sequence': 'Aleksander Gagarin był pierwszym człowiekiem w kosmosie'},
 {'score': 0.033182237297296524,
  'token': 22456,
  'token_str': 'Albert',
  'sequence': 'Albert Gagarin był pierwszym człowiekiem w kosmosie'},
 {'score': 0.032033007591962814,
  'token': 31195,
  'token_str': 'Charles',
  'sequence': 'Charles Gagarin był pierwszym człowiekiem w kosmosie'}]

Yuri by się zmartwił. Chociaż może to mało "polskie" zdanie:

In [77]:
herbert_fm("Mirosław <mask> był pierwszym i dotychczas jedynym Polakiem, który odbył lot w kosmos.")

[{'score': 0.12764084339141846,
  'token': 39710,
  'token_str': 'Drzewiecki',
  'sequence': 'Mirosław Drzewiecki był pierwszym i dotychczas jedynym Polakiem, który odbył lot w kosmos.'},
 {'score': 0.029964834451675415,
  'token': 24394,
  'token_str': 'Jabłoński',
  'sequence': 'Mirosław Jabłoński był pierwszym i dotychczas jedynym Polakiem, który odbył lot w kosmos.'},
 {'score': 0.027027016505599022,
  'token': 19421,
  'token_str': 'Wiśniewski',
  'sequence': 'Mirosław Wiśniewski był pierwszym i dotychczas jedynym Polakiem, który odbył lot w kosmos.'},
 {'score': 0.023340020328760147,
  'token': 26265,
  'token_str': 'Witkowski',
  'sequence': 'Mirosław Witkowski był pierwszym i dotychczas jedynym Polakiem, który odbył lot w kosmos.'},
 {'score': 0.020443130284547806,
  'token': 24285,
  'token_str': 'Michalski',
  'sequence': 'Mirosław Michalski był pierwszym i dotychczas jedynym Polakiem, który odbył lot w kosmos.'}]

Niestety nie.

In [107]:
herbert_fm("Najwyższa góra na świecie ma wysokość <mask> metrów ponad poziomem morza, natomiast najwyższy szczyt Europy, góra Elbrus, ma zaledwie 5,642 metrów ponad poziomem morza")

[{'score': 0.012200584635138512,
  'token': 4895,
  'token_str': '500',
  'sequence': 'Najwyższa góra na świecie ma wysokość 500 metrów ponad poziomem morza, natomiast najwyższy szczyt Europy, góra Elbrus, ma zaledwie 5, 642 metrów ponad poziomem morza'},
 {'score': 0.01208607479929924,
  'token': 7569,
  'token_str': '600',
  'sequence': 'Najwyższa góra na świecie ma wysokość 600 metrów ponad poziomem morza, natomiast najwyższy szczyt Europy, góra Elbrus, ma zaledwie 5, 642 metrów ponad poziomem morza'},
 {'score': 0.01175074651837349,
  'token': 48756,
  'token_str': '6000',
  'sequence': 'Najwyższa góra na świecie ma wysokość 6000 metrów ponad poziomem morza, natomiast najwyższy szczyt Europy, góra Elbrus, ma zaledwie 5, 642 metrów ponad poziomem morza'},
 {'score': 0.010753924958407879,
  'token': 3450,
  'token_str': '2000',
  'sequence': 'Najwyższa góra na świecie ma wysokość 2000 metrów ponad poziomem morza, natomiast najwyższy szczyt Europy, góra Elbrus, ma zaledwie 5, 642 metr

Też nie.

# Zadanie 6

Check zero-shot learning capabilites of the models. Provide at least 5 sentences with different sentiment for the following scheme: "'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie [MASK]" Try different prompts, to see if they make any difference.

Skoro `zero-shot` to te zdania podaję jako osobne prompty, nie łącznie

Czytając docsy `papuGaPT2` odniosłem wrażenie, że `zero-shot learning` to po prostu przechowywanie wiedzy przez model (screenshot poniżej)

![Zero-shot learning by papuga](image.png)

Co nieco kłóci mi się z bardziej znaną mi defnicją: 

> Zero-shot learning (ZSL) is a problem setup in deep learning where, at test time, a learner observes samples from classes which were not observed during training,
> and needs to predict the class that they belong to. Zero-shot methods generally work by associating observed and non-observed classes through some form of auxiliary information,
> which encodes observable distinguishing properties of objects.

(cytat z wiki)


Pójdę za definicją z wiki & zasugerowanym przykładem.

In [110]:
prompt_1 = "Poziom usług medycznych w przychodniach Scanmed jest wprost niesamowity. Wydźwięk tej wypowiedzi jest zdecydowanie"
papuga_tg(prompt_1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Poziom usług medycznych w przychodniach Scanmed jest wprost niesamowity. Wydźwięk tej wypowiedzi jest zdecydowanie pozytywny. Pacjenci mogą liczyć nie tylko na profesjonalne podejście wykwalifikowanych lekarzy, ale również na opiekę medyczną najwyższej jakości.\nNasza przychodnia oferuje konsultacje lekarzy specjalistów zajmujących'}]

In [111]:
herbert_fm(prompt_1 + " <mask>")

[{'score': 0.3778590261936188,
  'token': 28531,
  'token_str': 'pozytywny',
  'sequence': 'Poziom usług medycznych w przychodniach Scanmed jest wprost niesamowity. Wydźwięk tej wypowiedzi jest zdecydowanie pozytywny'},
 {'score': 0.08948273956775665,
  'token': 11470,
  'token_str': 'lepszy',
  'sequence': 'Poziom usług medycznych w przychodniach Scanmed jest wprost niesamowity. Wydźwięk tej wypowiedzi jest zdecydowanie lepszy'},
 {'score': 0.08056753873825073,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Poziom usług medycznych w przychodniach Scanmed jest wprost niesamowity. Wydźwięk tej wypowiedzi jest zdecydowanie.'},
 {'score': 0.045824114233255386,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Poziom usług medycznych w przychodniach Scanmed jest wprost niesamowity. Wydźwięk tej wypowiedzi jest zdecydowanie …'},
 {'score': 0.030652863904833794,
  'token': 17221,
  'token_str': 'wyższy',
  'sequence': 'Poziom usług medycznych w przychodniach Scanmed jest wprost niesam

Oba modele dały radę, ale może być tak, że fraza "wydźwięk tej wypowiedzi jest zdecydowanie" jest bardzo "sugerująca" dla modelu i odpowiednie słowo jest wstawione tylko na podstawie tego kontekstu, bez uwzględnienia "sentymentu" całej wypowiedzi.
Sprawdźmy to:

In [121]:
prompt_2 = "Poziom usług medycznych w przychodniach Scanmed jest wprost żenujący. Wydźwięk tej wypowiedzi jest zdecydowanie"
papuga_tg(prompt_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Poziom usług medycznych w przychodniach Scanmed jest wprost żenujący. Wydźwięk tej wypowiedzi jest zdecydowanie negatywny.\n„Jeśli się przyjmą, to będzie się ich tak długo uczył, dopóki będą miały jakieś kłopoty” – takie zdania padają najczęściej.'}]

In [122]:
herbert_fm(prompt_2 + " <mask>")

[{'score': 0.2659365236759186,
  'token': 39051,
  'token_str': 'negatywny',
  'sequence': 'Poziom usług medycznych w przychodniach Scanmed jest wprost żenujący. Wydźwięk tej wypowiedzi jest zdecydowanie negatywny'},
 {'score': 0.11085037887096405,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Poziom usług medycznych w przychodniach Scanmed jest wprost żenujący. Wydźwięk tej wypowiedzi jest zdecydowanie.'},
 {'score': 0.08360789716243744,
  'token': 25812,
  'token_str': 'gorszy',
  'sequence': 'Poziom usług medycznych w przychodniach Scanmed jest wprost żenujący. Wydźwięk tej wypowiedzi jest zdecydowanie gorszy'},
 {'score': 0.0739324688911438,
  'token': 13028,
  'token_str': 'zły',
  'sequence': 'Poziom usług medycznych w przychodniach Scanmed jest wprost żenujący. Wydźwięk tej wypowiedzi jest zdecydowanie zły'},
 {'score': 0.04786384850740433,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Poziom usług medycznych w przychodniach Scanmed jest wprost żenujący. Wydźwięk tej

9 / 10 uruchomień komórki z "papugą", odpowiedź była: "negatywny" <-- więc jednak można wysuwać nieśmiałe wnioski, że model potrafi przewidzieć tą wybraną klasę na podstawie kontekstu.

"herbert" poprawnie odczytywał moje intencje.

Popatrzmy na kolejne prompty:

In [123]:
# Wydźwięk "neutralny"
prompt_3 = "Trasa autobusu 159 rozpoczyna się w Cichym Kąciku, przebiega przez liczne przystanki, aby ostatecznie zakończyć swój bieg na Mistrzejowicach. Wydźwięk tej wypowiedzi jest zdecydowanie"

In [133]:
papuga_tg(prompt_3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Trasa autobusu 159 rozpoczyna się w Cichym Kąciku, przebiega przez liczne przystanki, aby ostatecznie zakończyć swój bieg na Mistrzejowicach. Wydźwięk tej wypowiedzi jest zdecydowanie negatywny – to znak, że trzeba zmienić formę aktywności obywatelskiej w naszym mieście.'}]

7/10 - pozytywny, pozostałe negatywny, brak odpowiedzi "neutralny" (bądź czegoś podobnego)

In [134]:
herbert_fm(prompt_3 + " <mask>")

[{'score': 0.17054495215415955,
  'token': 28531,
  'token_str': 'pozytywny',
  'sequence': 'Trasa autobusu 159 rozpoczyna się w Cichym Kąciku, przebiega przez liczne przystanki, aby ostatecznie zakończyć swój bieg na Mistrzejowicach. Wydźwięk tej wypowiedzi jest zdecydowanie pozytywny'},
 {'score': 0.13442207872867584,
  'token': 39051,
  'token_str': 'negatywny',
  'sequence': 'Trasa autobusu 159 rozpoczyna się w Cichym Kąciku, przebiega przez liczne przystanki, aby ostatecznie zakończyć swój bieg na Mistrzejowicach. Wydźwięk tej wypowiedzi jest zdecydowanie negatywny'},
 {'score': 0.1079329177737236,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Trasa autobusu 159 rozpoczyna się w Cichym Kąciku, przebiega przez liczne przystanki, aby ostatecznie zakończyć swój bieg na Mistrzejowicach. Wydźwięk tej wypowiedzi jest zdecydowanie.'},
 {'score': 0.07012031227350235,
  'token': 5868,
  'token_str': 'inny',
  'sequence': 'Trasa autobusu 159 rozpoczyna się w Cichym Kąciku, przebiega p

Tutaj też pozytywny / negatywny. Wygląda, że przewidywane są tylko dwie klasy. Może dodam więcej kontekstu, z przykładem dla trzeciej klasy (ale to sprawia, że w kontekście pojawia się więcej wypowiedzi i ciężej może być ocenić jej wydźwięk).

In [135]:
prompt_4 = prompt_3 +  " neutralny. Podobnie będzie, jeżeli powiemy, że trasa linii 20 rozpoczyna się w Cichym Kąciku i biegnie aż na Mały Płaszów. Wtedy wydźwięk tej wypowiedzi jest zdecydowanie"

In [137]:
papuga_tg(prompt_4, max_length=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Trasa autobusu 159 rozpoczyna się w Cichym Kąciku, przebiega przez liczne przystanki, aby ostatecznie zakończyć swój bieg na Mistrzejowicach. Wydźwięk tej wypowiedzi jest zdecydowanie neutralny. Podobnie będzie, jeżeli powiemy, że trasa linii 20 rozpoczyna się w Cichym Kąciku i biegnie aż na Mały Płaszów. Wtedy wydźwięk tej wypowiedzi jest zdecydowanie pozytywny.\nZważywszy na fakt faktu, że linie zostały zawieszone w ramach umowy z KPRP z dniem 31 marca 2016 r.'}]

In [138]:
herbert_fm(prompt_4 + " <mask>")

[{'score': 0.2704119086265564,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Trasa autobusu 159 rozpoczyna się w Cichym Kąciku, przebiega przez liczne przystanki, aby ostatecznie zakończyć swój bieg na Mistrzejowicach. Wydźwięk tej wypowiedzi jest zdecydowanie neutralny. Podobnie będzie, jeżeli powiemy, że trasa linii 20 rozpoczyna się w Cichym Kąciku i biegnie aż na Mały Płaszów. Wtedy wydźwięk tej wypowiedzi jest zdecydowanie …'},
 {'score': 0.14628084003925323,
  'token': 39051,
  'token_str': 'negatywny',
  'sequence': 'Trasa autobusu 159 rozpoczyna się w Cichym Kąciku, przebiega przez liczne przystanki, aby ostatecznie zakończyć swój bieg na Mistrzejowicach. Wydźwięk tej wypowiedzi jest zdecydowanie neutralny. Podobnie będzie, jeżeli powiemy, że trasa linii 20 rozpoczyna się w Cichym Kąciku i biegnie aż na Mały Płaszów. Wtedy wydźwięk tej wypowiedzi jest zdecydowanie negatywny'},
 {'score': 0.1226472333073616,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Trasa autobus

Znów tylko pozytywny / negatywny.

Więc o ile modele te mniej więcej potrafią odczytać te dwie klasy, to wymaganie rozpoznania nowej, wcześniej nie obserwowanej, to dla nich za dużo (przynajmniej w przypadku promptów które zaproponowałem).

In [139]:
prompt_5 = "Wykład profesora Iksińskiego był fascynujący, jestem zachwycony tym, że można opowiadać w tak angażujący słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie"

In [140]:
papuga_tg(prompt_5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Wykład profesora Iksińskiego był fascynujący, jestem zachwycony tym, że można opowiadać w tak angażujący słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie pozytywny. Wykład profesora Iksińskiego jest godny polecenia nauczycielom.\nBardzo dziękuję za recenzję tego wykładu'}]

In [142]:
herbert_fm(prompt_5 + " <mask>")

[{'score': 0.28318530321121216,
  'token': 28531,
  'token_str': 'pozytywny',
  'sequence': 'Wykład profesora Iksińskiego był fascynujący, jestem zachwycony tym, że można opowiadać w tak angażujący słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie pozytywny'},
 {'score': 0.10273756831884384,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Wykład profesora Iksińskiego był fascynujący, jestem zachwycony tym, że można opowiadać w tak angażujący słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie …'},
 {'score': 0.07327660918235779,
  'token': 11470,
  'token_str': 'lepszy',
  'sequence': 'Wykład profesora Iksińskiego był fascynujący, jestem zachwycony tym, że można opowiadać w tak angażujący słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie lepszy'},
 {'score': 0.05549129843711853,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Wykład profesora Iksińskiego był fascynujący, jestem zachwycony tym, że można opowiadać w tak angażujący słuchacza sposób. Wyd

Tutaj poprawnie odczytany sentyment

In [155]:
prompt_6 = "Wykład profesora Iksińskiego był do nieczego, jestem zdumiony tym, że można opowiadać w tak nudzący słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie"

In [156]:
papuga_tg(prompt_6)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Wykład profesora Iksińskiego był do nieczego, jestem zdumiony tym, że można opowiadać w tak nudzący słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie inny, jeśli w ogóle. Jeżeli nakreślimy na końcu języka jakiś wyob'}]

O, widzimy tutaj, że pozytywny / negatywny to nie jedyne co `papuga` potrafi wstawić

In [162]:
herbert_fm(prompt_6 + " <mask>")

[{'score': 0.12719668447971344,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Wykład profesora Iksińskiego był do nieczego, jestem zdumiony tym, że można opowiadać w tak nudzący słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie …'},
 {'score': 0.11666170507669449,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Wykład profesora Iksińskiego był do nieczego, jestem zdumiony tym, że można opowiadać w tak nudzący słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie.'},
 {'score': 0.10588186234235764,
  'token': 5868,
  'token_str': 'inny',
  'sequence': 'Wykład profesora Iksińskiego był do nieczego, jestem zdumiony tym, że można opowiadać w tak nudzący słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie inny'},
 {'score': 0.09265244752168655,
  'token': 39051,
  'token_str': 'negatywny',
  'sequence': 'Wykład profesora Iksińskiego był do nieczego, jestem zdumiony tym, że można opowiadać w tak nudzący słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydow

Tutaj odpowiedź jest ok. 

Jeszcze popatrzmy na wypowiedź mieszaną:

In [164]:
prompt_7 = "Wykład profesora Iksińskiego był do nieczego, jestem zachwycony tym, że można opowiadać w tak obojętny dla słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie"

In [170]:
papuga_tg(prompt_7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Wykład profesora Iksińskiego był do nieczego, jestem zachwycony tym, że można opowiadać w tak obojętny dla słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie do przyjęcia, chociaż w przypadku języka mówionego jest on trochę bardziej wyważony.'}]

Tutaj model najczęściej nie klasyfikował na `pozytywny negatywny` tylko uzupełniał inną fantazją.

In [171]:
herbert_fm(prompt_7 + " <mask>")

[{'score': 0.1544850617647171,
  'token': 1826,
  'token_str': '…',
  'sequence': 'Wykład profesora Iksińskiego był do nieczego, jestem zachwycony tym, że można opowiadać w tak obojętny dla słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie …'},
 {'score': 0.13079506158828735,
  'token': 28531,
  'token_str': 'pozytywny',
  'sequence': 'Wykład profesora Iksińskiego był do nieczego, jestem zachwycony tym, że można opowiadać w tak obojętny dla słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie pozytywny'},
 {'score': 0.08461660146713257,
  'token': 5868,
  'token_str': 'inny',
  'sequence': 'Wykład profesora Iksińskiego był do nieczego, jestem zachwycony tym, że można opowiadać w tak obojętny dla słuchacza sposób. Wydźwięk tej wypowiedzi jest zdecydowanie inny'},
 {'score': 0.0798211544752121,
  'token': 1899,
  'token_str': '.',
  'sequence': 'Wykład profesora Iksińskiego był do nieczego, jestem zachwycony tym, że można opowiadać w tak obojętny dla słuchacza sposób. Wy

"inny" / "pozytywny" / "lepszy" - model  sam nie może się zdecydować, co pokrywa się z wydźwiękiem samej wypowiedzi :D 

# Zadanie 8

Answer the following questions (2 points):

1. Which of the models produced the best results?

Nie prowadziłem, żadnych statystyk, ale bardzo subiektywnie wskazuję na `herBERT`. Zdaje się, że nieco lepiej wykrywał sentyment.
Na moją ocenę może mieć wpływ również to, że `herBERT` miał możliwość korzystania z `fill-mask` a "papuga" nie. Może warto było również sprawdzić jak `herBERT` radziłby sobie
z uzupełnianiem maski wewnątrz wypowiedzi, a nie tylko na jej skraju.


2. Was any of the models able to capture Polish grammar?

Tak, obydwa modele potrafiły dopasować swoją "fantazję" (wygenerowany output) do kontekstu wypowiedzi, aby ta była poprawna gramatycznie -- a przynajmniej bezpośrednia kontynuuacja. Im większy rozmiar fantazji (outputu), tym łatwiej było o błędy i zgubienie kontekstu.
Problemy miałem jedynie z wołaczem, ale tutaj bardzo prawdopodobne, że moje prompty nie były najlepsze.


3. Was any of the models able to capture long-distant relationships between the words?

Tak, testowałem tutaj przykłady z płcią podmiotu i dopasowaniem odpowiedniej formy czasownika ->  obydwa modele, w ramach tych krótkich wypowiedzi, którymi je raczyłem, potrafiły doposować się do kontekstu.
Podobnie jak w 2: zdarzało się, że dla dłuższej fantazji ta wiedza była już gubiona + w przypadku "papugi" skuteczność była mniejsza (mniejszy odsetek poprawnie gramatycznych odpowiedzi).

Zwracam też uwagę, że często o ile same wypowiedzi były jako tako poprawne gramatycznie, to "sensu" nie miały.


4. Was any of the models able to capture world knowledge?

W przypadku obu modeli, dla moich pytań, żaden z nich nie podał poprafnych faktograficznie odpowiedzi. Możliwe, że moje pytania były na tyle specyficzne, że korpus na którym były trenowane nie zawierał tych danych. 
Z tego co widziałem w dokumentacji "papugi" to potrafi ona zwrócić sensowny output dla pytań typu: "najlepszym polskim poetą był". Dla `herBERT` nie mogę wysnuć takich wniosków.

5. Was any of the models good at doing zero-shot classification?

Zwracam uwagę, że użyta przeze mnie fraza `wydźwięk tej wypowiedzi jest zdecydowanie` jest specyficzna -- intuicyjnie bardzo często występuje razem (mieli byśmy tutaj wysokie PMI da 3/4-gramów), więc ta klasyfikacja nieznanych wcześniej kategorii może być tutaj zaburzona.
W przypadku wyraźnych "sentymentów" w wypowiedzi oba modele radziły sobie dobrze (`herBERT` lepiej). W przypadku ambiwalentnych wypowdzi, bądź po prostu neutralnych było gorzej w przypadku obu modeli.

6. What are the most striking errors made by the models?

Przedewszystkim faktograficzne błędy (widoczne w wynikach wyżej), zwracałem na nie uwagę na bieżąco.
"Papuga" też bardzo często "bredziła" zupełnie bez sensu, choć poprawnie gramatycznie. W przypadku `herBERT`a tego nie obserwowałem, ale korzystałem z niego jedynie w trybie `fill-mask`, nie generując dłuższych fantazji.